# Description Based Recommender

In [4]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer
import glob
from collections import Counter
from string import punctuation

#Import data from the clean file 
product_df = pd.read_csv('product_info_clean.csv', index_col = 0)

#Print the head of the cleaned DataFrame
product_df.head()

,id,name,review_count,rating_average,price,description,category,category_name
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,"Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...",914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,bình thông hơi dành cho xe máy tặng bao tay xe...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,"Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...",914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,"Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...",49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...


In [5]:
stop_word = []
with open("stop_word.txt",encoding="utf-8") as f:
    text = f.read()
    for word in text.split() :
        stop_word.append(word)
    f.close()
punc = list(punctuation)
stop_word = stop_word + punc
print(stop_word)

['bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 'có', 'có_thể', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến_nỗi', 'đều', 'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một_cách', 'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi', 'nữa', 'phải', 'qua', 'ra', 'rằng', 'rằng', 'rất', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên', 'trước', 'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa', '!', '"', '#', '$', '%', '&', "'", ',', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\\\', ']', '^', '_', '`', '{', '|', '}', '~', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']


In [6]:
def clean_text(text):
    text_lower = text.lower()
    text_token = ViTokenizer.tokenize(text_lower)
    
    sent = []
    for word in text_token.split("_\s") :
        if (word not in stop_word) :
            if ("_" in word) or (word.isalpha() == True):
                sent.append(word)
    return " ".join(sent)

product_df['description_token'] = product_df['description'].apply(clean_text)
product_df.head()

,id,name,review_count,rating_average,price,description,category,category_name,description_token
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...,"hỗ_trợ dòng lên đến 2.4 a , cho tốc_độ sạc nha..."
1,44105732,"Dép sục unisex duet độn đế, chống trượt Litic ...",27,3.5,148500,"Tiện lợi, bền đẹp, siêu nhẹ Độn đế tinh tế, kh...",914,Giày - Dép nữ/Giày lười nữ/Giày lười hở gót,"tiện_lợi , bền đẹp , siêu nhẹ độn đế tinh_tế ,..."
2,54132740,bình thông hơi dành cho xe máy tặng bao tay xe...,4,4.5,199000,bình thông hơi dành cho xe máy tặng bao tay xe...,6061,Ô Tô - Xe Máy - Xe Đạp/Phụ kiện - Chăm sóc xe/...,bình thông hơi dành cho xe_máy tặng bao_tay xe...
3,65011722,"Đồng Hồ Nam FNGEEN FE230 mặt đồng hồ tròn, th...",19,4.5,179000,"Mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng ...",914,Đồng hồ và Trang sức/Đồng hồ nam/Đồng hồ busin...,"mặt đồng_hồ tròn , thiết_kế đẹp_mắt , sáng bón..."
4,7995403,Giày Sneaker Unisex Converse Chuck Taylor All ...,19,5.0,1500000,"Đường may vô cùng tỉ mỉ, tinh tế Chất vải canv...",49620,Giày - Dép nam/Giày thể thao nam/Giày thể thao...,"đường may vô_cùng tỉ_mỉ , tinh_tế chất vải can..."


In [7]:
product_df['name'].iloc[3]

'Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay '

In [8]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(min_df=5,max_df= 0.8,max_features=3000,sublinear_tf=True)

#Replace NaN with an empty string
product_df['description_token'] = product_df['description_token'].fillna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the description_token feature
tfidf_matrix = tfidf.fit_transform(product_df['description_token'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(21838, 3000)

In [9]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
#Construct a reverse mapping of indices and product ids, and drop duplicate ids, if any
indices = pd.Series(product_df.index, index=product_df['id']).drop_duplicates()

In [11]:
# Function that takes in product id as input and gives recommendations 
def content_recommender(id, cosine_sim=cosine_sim, product_df=product_df, indices=indices):
    # Obtain the index of the product that matches the title
    idx = indices[id]

    # Get the pairwsie similarity scores of all products with that product
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the products based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 20 most similar products. Ignore the first product.
    sim_scores = sim_scores[1:21]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    # Return the top 20 most similar products
    return product_df['name'].iloc[product_indices]

In [12]:
#Get recommendations for Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay 
donghonam_id = product_df[product_df['name'].str.find('Đồng Hồ Nam FNGEEN FE230  mặt đồng hồ tròn, thiết kế đẹp mắt, sáng bóng với tính năng hiện đại cho phái mạnh tự tin, mạng mẽ và thời trang  Dây thép không gỉ thiết kế ôm tay ') == 0]['id'].iloc[0]
content_recommender(donghonam_id)

7249     Đồng Hồ Nam FNGEEN FE708 mạ vàng mặt đồng hồ t...
11967    Đồng Hồ Nam FNGEEN FG0541 doanh nhân 2020 Lịch...
17822    Đồng hồ nam FNGEEN 5055 lịch ngày chính hãng n...
17906    Đồng Hồ Nam FNGEEN F281 mặt số đính đá Lịch Ng...
3290     Đồng hồ nam dây da thời trang - đồng hồ thể th...
17081    Đồng hồ nam ECONOMICXI dây thép mành đen chạy ...
13636    Đồng Hồ Nam Fourron F66 Lịch Ngày Dây Thép Cao...
15295    Đồng hồ nam chống nước NO6216G, phong cách trẻ...
2586     Đồng Hồ Nam Dây Thép Cao Cấp Thời Trang Lịch L...
477      Đồng hồ nam FNGEEN fn668 Mặt rồng 3d cuộn hình...
4799     Đồng Hồ Nam Fourron Lịch Ngày chính hãng nam s...
4918     Đồng Hồ Nam 2 Lịch Crnaira CR049 Dây Thép Mành...
6188     Đồng Hồ Nam Crnaira CR8333 Siêu Mỏng mặt đồng ...
10334    Đồng hồ Nam CUENA CUN8207 Lịch ngày sang trọng...
9980     Đồng hồ nam HALEI 564 mặt vuông thời trang cao...
9049     Đồng Hồ Nam Chạy 6 Kim Crnaira PE90 Mặt Đính Đ...
5144     Đồng Hồ Nam 6 Kim Crnaira C099 Dây Hợp Kim Thé.

In [13]:
import pickle
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print("Save file: ", filename)
save_pickle(cosine_sim, 'cosine_sim.pkl')

Save file:  cosine_sim.pkl


# Collaborative Filtering

## The Framework

In [16]:
import pandas as pd
import numpy as np

In [22]:
#Load the review file into a dataframe
orig_ratings = pd.read_csv('product_review_clean.csv', index_col = 0)
orig_ratings.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,customer_id,product_id,rating,thank_count,title,content,created_at,purchase_at,customer_name
0,5719606,21299202,5,6,"Dây 2m, chất liệu của dây dù",Xài được khoảng 1 tuần thì có vẻ ổn. Dây dài 2...,2020-08-25 14:04:15,2020-07-01 13:22:12,Khách Hàng
1,11891537,21299202,1,19,PSK Dragon giao láo bố láo.,"Đặt 3 sợi màu đỏ dài 2m , giao 2 sợi màu đen 2...",2020-08-12 11:41:41,2020-08-06 22:05:10,Trung Tung Tăng
2,7630980,21299202,4,0,Được,"cảm nhận ban đầu thì dây rất cứng, \ndây đúng ...",2020-09-04 11:15:21,2020-09-02 21:39:54,Nguyễn Tất Thắng
3,1242105,21299202,1,0,Rất không hài lòng,"Chất lượng hoàn thiện kém, kích thước chân lig...",2020-08-15 14:19:43,2020-08-13 20:50:11,Doan Hiep
4,10815590,21299202,4,8,Ổn,Hoàn thiện khá tốt\nNhưng tại sao dùng bút thử...,2020-03-08 13:23:01,2020-02-21 16:38:52,Hoàng việt hưng


In [23]:
ratings = orig_ratings.sample(n=10000, random_state =40)

In [24]:
#Get the unnecessary column
features = ['customer_id', 'product_id', 'rating']
ratings  = ratings[features]
ratings.head()

,customer_id,product_id,rating
680634,5764785,63171394,5
156432,19028885,20585233,4
363368,5996665,9856777,5
629209,6767270,36626940,5
44921,7450531,557981,1


In [25]:
#Import the train_test_split function
from sklearn.model_selection import train_test_split

#Assign X as the original ratings dataframe and y as the customer_id column of ratings.
X = ratings.drop('rating', axis=1)
y = ratings['rating']

#Split into training and test datasets, stratified along customer_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

In [26]:
#Import the mean_squared_error function
from sklearn.metrics import mean_squared_error

#Function that computes the root mean squared error (or RMSE)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [27]:
#Define the baseline model to always return 3.
def baseline(user_id, movie_id):
    return 3.0

In [28]:
#Function to compute the RMSE score obtained on the testing set by a model
def score(cf_model):
    
    #Construct a list of user-product tuples from the testing dataset
    id_pairs = zip(X_test['customer_id'], X_test['product_id'])
    
    #Predict the rating for every user-product tuple
    y_pred = np.array([cf_model(user, product) for (user, product) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(y_test)
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [29]:
score(baseline)

1.7731328207441202

## Model Based Approaches

In [30]:
from sklearn.svm import SVR

model = SVR(kernel='rbf').fit(X_train, y_train)

In [31]:
from sklearn.metrics import mean_squared_error
mean_squared_error(model.predict(X_test), y_test)

1.1026685663261666

In [32]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(model.predict(X_test), y_test)

0.5524651467637168

The RMSE score is very good (close to 1)
# Example usage: Recommend products for user 

In [33]:
def collaborative_recommender(customer_id):
    #Compute the predicted ratings using the SVR filter
    product_ids = product_df['id'].unique()
    id_pairs = np.array([(customer_id, product_id) for product_id in product_ids])
    ratings_est = model.predict(id_pairs)
    products = pd.Series(ratings_est, index=product_ids)
    #Sort the products in decreasing order of predicted rating
    products = products.sort_values(ascending=False)
    
    #Return the top 10 products as recommendations
    return product_df[product_df['id'].isin(products.head(10).index)]

## User ID 1874401 : Nguyen Trung

In [34]:
customer_id = 1874401

Get customer name

In [35]:
orig_ratings[orig_ratings['customer_id']==customer_id].iloc[0,[0,8]]

customer_id           1874401
customer_name    Nguyen Trung
Name: 11, dtype: object

### Previous bought products

In [36]:
product_df[product_df['id'].isin(orig_ratings[orig_ratings['customer_id'] == customer_id]['product_id'])].head()

,id,name,review_count,rating_average,price,description,category,category_name,description_token
0,21299202,Cáp sạc và truyền dữ liệu tốc độ cao Baseus Ho...,216,4.5,67000,"Hỗ trợ dòng lên đến 2.4 A , cho tốc độ sạc nha...",1815,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...,"hỗ_trợ dòng lên đến 2.4 a , cho tốc_độ sạc nha..."
1392,10697849,TotoLink N150UA - USB Wi-Fi Chuẩn N 150Mbps - ...,32,4.5,111000,Tiêu chuẩn: IEEE 802.11 b/g/n Tốc độ 150Mbps K...,4294,Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Mạng...,tiêu_chuẩn : ieee 802.11 b / g / n tốc_độ 150m...
3607,6772415,Lò Nướng Điện Sunhouse SHD4206 (10L) - Hàng ch...,1069,4.5,359000,Dung tích: 10L Công suất: 800W Sản phẩm này có...,1882,Điện Gia Dụng/Đồ dùng nhà bếp/Lò nướng điện/Lò...,dung_tích : 10l công_suất : 800w sản_phẩm này ...
5828,356371,Ổ Cứng Di Động Transcend StoreJet M3S 1TB USB ...,138,4.5,1393500,Loại ổ cứng: ổ cứng di động Kết nối: USB 2.0/U...,1827,Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Lưu ...,loại ổ_cứng : ổ_cứng di_động kết_nối : usb 2.0...
8253,19696906,"Bộ 5 Dây Ngũ Sắc Đàn Hồi Tập Thể Hình, Tập Gym...",114,4.5,175000,Sản phẩm đa năng tập được tất cả các nhóm cơ M...,1975,Thể Thao - Dã Ngoại/Dụng cụ - thiết bị tập thể...,sản_phẩm đa_năng tập được tất_cả các nhóm cơ m...


### Recommended products

In [37]:
collaborative_recommender(customer_id)

,id,name,review_count,rating_average,price,description,category,category_name,description_token
6840,35355630,"Dép nam, nữ xỏ ngón thời trang T334K",40,4.5,73000,"Dép làm từ chất liệu cao su cao cấp, chắc chắn...",914,Giày - Dép nam/Dép nam/Dép nam xỏ ngón,"dép làm từ chất_liệu cao_su cao_cấp , chắc_chắ..."
6959,35355941,"Dép nam, nữ quai ngang thời trang T335K",28,4.5,77000,"Dép làm từ chất liệu xốp eva cao cấp, chắc chắ...",914,Giày - Dép nam/Dép nam/Dép nam quai ngang,"dép làm từ chất_liệu xốp eva cao_cấp , chắc_ch..."
7767,35358455,HealthFit – Gói tập 12 tháng tất cả các môn Yo...,0,0.0,2110000,TRẢI NGHIỆM CHƯƠNG TRÌNH HỌC PT GX (1 kèm 1) C...,11312,Voucher - Dịch vụ/Spa & Làm đẹp/Phòng tập/GYM,trải nghiệm chương_trình học pt gx ( 1 kèm 1 )...
9815,35364467,Cặp gài kính silicon chống rơi rớt tiện lợi,159,4.5,28000,Nhẹ và thoải mái. Dẻo bền. Có thể sử dụng hàng...,914,Phụ kiện thời trang/Mắt kính/Phụ kiện kính,nhẹ và thoải_mái . dẻo bền . có_thể sử_dụng hà...
10704,35367014,"Thảm Tập YoGa, Gym miDoctor + Bao Thảm Tập Yog...",492,4.5,155000,Thảm Tập Yoga Chính Hãng miDoctor + Bao Thảm T...,1975,Thể Thao - Dã Ngoại/Các môn thể thao khác/Yoga...,thảm tập yoga chính hãng midoctor + bao thảm t...
11194,35368490,Chảo đáy từ vân đá chống dính GREEN COOK...,47,4.5,155000,Chống dính ưu việt Sử dụng được cho bếp...,1883,Nhà Cửa - Đời Sống/Dụng cụ nhà bếp/Dụng cụ nấu...,chống dính ưu_việt sử_dụng được cho bếp từ an_...
12662,35373097,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,58,3.5,165000,Tai nghe bluetooth AMOI F9 mang lại trải nghiệ...,1811,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,tai_nghe bluetooth amoi f9 mang lại trải nghiệ...
13093,35374344,Giày Thể Thao Nữ Sneaker Tăng Chiều Cao Độn Đế...,8,3.0,179000,Đế được làm bằng chất liệu cao su tổng ...,2,Giày - Dép nữ/Giày thể thao nữ/Giày thể thao c...,"đế được làm bằng chất_liệu cao_su tổng_hợp , c..."
13149,35374545,Túi Đeo Chéo Nữ Túi Xách Tay Nữ Thời Trang Đôn...,3,3.0,99000,Chất liệu: PU Kiểu: họa tiết trơn Phong cách: ...,2,"Túi thời trang nữ/Túi đeo chéo, túi đeo vai nữ...",chất_liệu : pu kiểu : họa tiết trơn phong_cách...
13696,35376243,Cáp micro-USB sạc nhanh và truyền dữ liệu ZCC-...,75,4.5,29000,Cổng kết nối: micro-USB Cáp sạc nhanh (2.4A) v...,2,Thiết Bị Số - Phụ Kiện Số/Phụ Kiện Điện Thoại ...,cổng kết_nối : micro - usb cáp sạc nhanh ( 2.4...
